### import

In [1]:
!python --version

Python 3.10.12


In [2]:
!pip install numpy==1.23.1
!pip install gluonnlp==0.10.0
!pip install pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch
!pip install pandas

  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661750 sha256=d30b74a65b0eae86ba751598fbf54c1fe053e6558606cd73bd89fe25dfc79b7d
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 11.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: me

In [3]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3


## 코드

In [4]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-p5ehuit6/kobert-tokenizer_81b51230b1da44d1b13a333ad8634e53
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-p5ehuit6/kobert-tokenizer_81b51230b1da44d1b13a333ad8634e53
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=d7aa5d5261ea3db1d5adb936eee82e089c1beeed7bdc9fb20df9da8fff91e8a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-v6ppnxn1/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = '1'

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용시
device = torch.device("cuda:0")
# device = torch.device("cpu")

def get_kobert_model(model_path, vocab_file, ctx="cpu"):
  bertmodel = BertModel.from_pretrained(model_path)
  device = torch.device(ctx)
  bertmodel.to(device)
  bertmodel.eval()
  vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file, padding_token='[PAD]')
  return bertmodel, vocab_b_obj

#koBERT
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1")
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [7]:
tokenizer.encode("한국어 모델을 공유합니다.")

[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [8]:
# get dataset
datasetURL = "https://raw.githubusercontent.com/IDIOcoder/Chat-bot/main/sentence_emotion.csv"
data = pd.read_csv(datasetURL, encoding='cp949')

data.loc[(data['emotion']=="happiness"),'emotion'] = 0 #행복 -> 0
data.loc[(data['emotion']=="neutral"), 'emotion'] = 1 #중립 -> 1
data.loc[(data['emotion']=="sadness"), 'emotion'] = 2 #슬픔 -> 2
data.loc[(data['emotion']=="anger"), 'emotion'] = 3 #분노 -> 3
data.loc[(data['emotion']=="unrest"), 'emotion'] = 4 #불안 -> 4
data.loc[(data['emotion']=="surprise"), 'emotion'] = 5 #놀람 -> 5
data.loc[(data['emotion']=="tired"), 'emotion'] = 6 #피곤 -> 6
data.loc[(data['emotion']=="regret"), 'emotion'] = 7 #후회 -> 7

data_list = []
for ques, label in zip(data['sentence'], data['emotion']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

In [9]:
#데이터가 잘 저장되었는지 확인
print(len(data_list))
print(data_list[0])

7561
['10분마다 연락을 해봤지. 근데 아직도 이녀석, 온다고만 하지 오질 않고 있어.', '3']


### 클래스 가중치 계산

In [10]:
from collections import Counter

# 데이터셋의 라벨을 모두 모아 클래스 빈도 계산
labels = [int(label) for _, label in data_list]
class_counts = Counter(labels)
num_samples = len(labels)

# 클래스 가중치 계산 (클래스 빈도의 역수)
class_weights = {cls: num_samples / count for cls, count in class_counts.items()}
class_weights_tensor = torch.tensor([class_weights[i] for i in sorted(class_weights.keys())], dtype=torch.float).to(device)

### 입력 데이터셋 토큰화

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [20]:
#Setting Parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [21]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

### KoBERT 모델 구현

In [22]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size=768,
               num_classes=8,
               dr_rate=None,
               params=None):
    super(BERTClassifier, self).__init__()
    self.bert=bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v]=1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

In [23]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params':[p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params':[p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor) #다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

train_dataloader

### 훈련

In [24]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))


<ipython-input-24-0cd8b201d8af>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/189 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.1490511894226074 train acc 0.09375
epoch 1 train acc 0.4679232804232804


<ipython-input-24-0cd8b201d8af>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/48 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7394386574074074


  0%|          | 0/189 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.15945211052894592 train acc 0.96875
epoch 5 train acc 0.94130291005291


  0%|          | 0/48 [00:00<?, ?it/s]

epoch 5 test acc 0.7303240740740741


In [25]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("행복이")
            elif np.argmax(logits) == 1:
                test_eval.append("중립이")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 4:
                test_eval.append("불안이")
            elif np.argmax(logits) == 5:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 6:
                test_eval.append("피곤이")
            elif np.argmax(logits) == 7:
                test_eval.append("후회가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [27]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 자고있는데 얼굴위로 바퀴벌레가 지나갔어
>> 입력하신 내용에서 놀람이 느껴집니다.


하고싶은 말을 입력해주세요 : 괜히 주식을 팔았나봐
>> 입력하신 내용에서 후회가 느껴집니다.


하고싶은 말을 입력해주세요 : 나 이벤트에 당첨됬어!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 지갑을 잃어버린거 같아.
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 꽁꽁 얼어붙은 한강위로 고양이가 걸어다닙니다.
>> 입력하신 내용에서 놀람이 느껴집니다.


하고싶은 말을 입력해주세요 : 그 고양이는 스코티시 폴드 였어
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 김치찌개 레시피 알려줘
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 커피사려고 줄서있었는데, 새치기 당했어
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 내 자전거 누가 엄복동해가진 않았겠지.?
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 내 자전거 누가 훔쳐가진 않았겠지?
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 내 자전거 누가 훔쳐갔을까?
>> 입력하신 내용에서 후회가 느껴집니다.


하고싶은 말을 입력해주세요 : 혹시 이거 죽을 병은 아니겠지?
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘 야근하고 왔어
>> 입력하신 내용에서 피곤이 느껴집니다.


하고싶은 말을 입력해주세요 : 0
